# Gradinet Checking
Backpropagation is quite challenging to implement, and sometimes has bugs, Sometimes due to creticality of the application we wants to be really certain that our implementation of backpropagation is correct. To have this reassurance we need to use "gradient checking".

In [1]:
# Imports
import numpy as np
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector, gradient_check_test_params

## One Dimentional Gradient Checking

In [2]:
# 1 Dimensonal Gradient Checking
def forward_propagation(x, theta):
    J = np.dot(theta, x)
    return J

In [3]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


In [4]:
def backward_propagation(x, theta):
    dtheta = x
    return x

In [5]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


In [6]:
def gradient_check(x, theta, epsilon = 1e-7):
    theta_plus = theta + epsilon
    theta_minus = theta - epsilon
    J_plus = np.dot(theta_plus, x)
    J_minus = np.dot(theta_minus , x)
    grad_approx = (J_plus - J_minus) / (2*epsilon)

    grad = x

    difference = np.linalg.norm(grad_approx - grad) / (np.linalg.norm(grad_approx) + np.linalg.norm(grad))

    if difference < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference

In [7]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

The gradient is correct!
difference = 2.919335883291695e-10


## N-Dimentional Gradient Checking

In [21]:
# Forward propogation for N-Dimentional Gradient Checking
def forward_propagation_n(X, Y, params):

    m = X.shape[1]
    W1 = params["W1"]
    b1 = params["b1"]
    W2 = params["W2"]
    b2 = params["b2"]
    W3 = params["W3"]
    b3 = params["b3"]

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)

    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)

    return cost , cache
     


In [11]:
# Backward propogation for N-Dimentional Gradient Checking
def backward_propagation_n(X, Y, cache):
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache

    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis = 1, keepdims=True)

    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims=True)

    grads = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return grads

In [ ]:
# Gradient Checking for N - Dimention
def gradient_check_n(params, grads, X, Y, epsilon = 1e-7):

    param_values, _ = dictionary_to_vector(params)
    grad_values = gradients_to_vector(grads)
    num_params = param_values.shape[0]
    J_plus = np.zeros((num_params, 1))
    J_minus = np.zeros((num_params, 1))
    grad_approx = np.zeros((num_params, 1))

    for i in range(num_params):

        theta_plus = np.copy(param_values)
        theta_plus[i][0] = theta_plus[i][0] + epsilon
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(theta_plus))

        theta_minus = np.copy(param_values)
        theta_minus[i][0] = theta_minus[i][0] - epsilon
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(theta_minus))

        grad_approx[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
    
    difference = (np.linalg.norm(grad_values - grad_approx)) / (np.linalg.norm(grad_values) + np.linalg.norm(grad_approx))
    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [26]:
X, Y, parameters = gradient_check_test_params()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

There is a mistake in the backward propagation! difference = 0.8075276040256201
